![title](http://www.networknuts-web.biz/wp-content/uploads/2017/02/ceph-cluster-monitoring-networknuts.png)

# Use Case - Directory Monitor

# Directory monitor

- Now that we know how to send an email from Python, let's use it for something useful
- E.g. let's use it to monitor changes in a directory, namely send a mail to yourself as soon as a new file is created in the directory

# Some useful libraries

- Three very useful libraries when you need to do some stuff with files and/or folders are **os**, **sys**, and **shutil**
- A few useful things you can do:
    - Creating a file path that works on all operating systems: `os.path.join(part1, part2, ... ,partn)`
    - Checking if a path exists: `os.path.exists(path)`
    - Checking if a path if to a *file*: `os.path.isfile(path)`
    - Checking if a path is to a *directory*: `os.path.isdir(path)`
    - Deleting a *file*: `os.remove(file_path)`
    - Deleting a *directory and all sub-directories and files*: `shutil.rmtree(path)`
    - Checking when a directory or file was last edited: `os.path.getctime(path)`
    - Listing all files in a directory: `os.listdir(path)`
    - Moving files: `os.rename(from_path, to_path)` / `shutil.move(from_path, to_path)`

# Importing stuff that we need

In [17]:
from examples.gmail_mail_sending import init_service, create_message, send_message
import os
from time import time

## Defining the two types of update

In [21]:
def get_updates_as_text(old_files, updated_files):
    removed_files = []
    new_files = []
    for f in old_files:
        if f not in updated_files:
            removed_files.append(f)
    for f in updated_files:
        if f not in old_files:
            new_files.append(f)
    text = ""
    if removed_files:
        text += "Removed files:\n{}\n\n".format("\n".join(removed_files))
    if new_files:
        text += "New files:\n{}".format("\n".join(new_files))
    return text

print(get_updates_as_text(["yolo.txt", "yo.txt"], ["hello.txt", "yo.txt"]))

Removed files:
yolo.txt

New files:
hello.txt


# Let's create the monitor

In [20]:
# Init the gmail service
gmail_service = init_service()

# Message parameters
sender, to, subject = "peter.sandberg@visma.com", "peter.sandberg@visma.com", "Directory updated!"

# Record the start time of monitoring
start_time = time()

# Define how long to run the monitor (in seconds)
max_time = 60

# Define the directory to monitor
dir_path = "data"

# All files in the directory at the start
old_files = os.listdir(dir_path)

while time() - start_time < max_time:
    updated_files = os.listdir(dir_path)
    if sorted(old_files) != sorted(updated_files):
        message_text = get_updates_as_text(old_files, updated_files)
        message = create_message(sender, to, subject, message_text)
        send_message(gmail_service, message)
        old_files = updated_files

KeyboardInterrupt: 